In [1]:
#Cargar la extensión autoreload
%load_ext autoreload
%autoreload 2
# Retriever tools
from libs.chatbot import Chatbot
from libs.vector_retriever import VectorRetriever
from libs.lexical_retriever import LexicalRetriever
from libs.basic_document_storage import BasicStorage
from libs.hybrid_retriever import HybridRetriever
from libs.pdf_loader import PDFLoader
from libs.basic_text_splitter import BasicTextSplitter

# Prompt Tools
from prompts.academic_assistant_prompt import system_prompt as academic_system_prompt
from prompts.academic_assistant_prompt import user_prompt as academic_user_prompt

In [2]:
# 1. Load, chunk and index the contents of the blog to create a retriever.
# Load the documents
pdf_loader = PDFLoader(file_filter={".pdf"})
documents = pdf_loader.load_data()
# Split the documents in chunks
splitter = BasicTextSplitter()
chunked_documents = splitter.split_documents(documents)
# Store the documents
basic_storage = BasicStorage()
basic_storage.add_documents(chunked_documents)


In [3]:
# 2. Create a retriever to handle the retrieval process.
# For handling the storage and retrieval of data chunks with semantic search
semantic_retriever = VectorRetriever()
# For handling the storage and retrieval of data chunks with lexical search
lexical_retriever = LexicalRetriever()
# A retriever for handling all the retrieval process
hybrid_retriever = HybridRetriever(storage_= basic_storage, retrievers=[semantic_retriever,lexical_retriever])
# Add the documents to the retriever
print("Adding documents to the retriever")
hybrid_retriever.add(chunked_documents)


Adding documents to the retriever


WARNING clustering 659 points to 25 centroids: please provide at least 975 training points


BM25S Create Vocab:   0%|          | 0/659 [00:00<?, ?it/s]

BM25S Convert tokens to indices:   0%|          | 0/659 [00:00<?, ?it/s]

BM25S Count Tokens:   0%|          | 0/659 [00:00<?, ?it/s]

BM25S Compute Scores:   0%|          | 0/659 [00:00<?, ?it/s]

In [7]:
import streamlit as st
bot = Chatbot(hybrid_retriever)

2024-12-25 15:09:29.289 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [5]:
query = "Cuales son las distintas formas de turismo"

In [9]:
bot.reset()
bot.reply(query)


2024-12-25 15:10:05.941 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

2024-12-25 15:10:06.725 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


ChatCompletionOutputMessage(role='assistant', content='Las formas de turismo incluyen:\n\n1. Turismo de sol y playa (relajación y actividad).\n2. Turismo cultural (ciudades y entornos culturales significativos).\n3. Turismo de reuniones (congresos, convenciones y otros eventos).\n4. Turismo deportivo (activo: torneos y competencias; pasivo: espectadores).\n5. Turismo náutico (buceo, vida a bordo, paseos en embarcaciones, yatismo, veleros, etc.).\n6. Turismo académico y científico (cursos, idiomas, etc.).\n7. Turismo de naturaleza (ecoturismo, agroturismo, vacaciones en la naturaleza).\n8. Turismo de salud (belleza, salud mental y corporal, hospitalario y no hospitalario).\n9. Turismo de negocios.\n10. Viajes de incentivo.\n11. Turismo de cruceros.\n12. Turismo de intereses sociales.\n13', tool_calls=None)


2024-12-25 15:10:07.191 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


'Las formas de turismo incluyen:\n\n1. Turismo de sol y playa (relajación y actividad).\n2. Turismo cultural (ciudades y entornos culturales significativos).\n3. Turismo de reuniones (congresos, convenciones y otros eventos).\n4. Turismo deportivo (activo: torneos y competencias; pasivo: espectadores).\n5. Turismo náutico (buceo, vida a bordo, paseos en embarcaciones, yatismo, veleros, etc.).\n6. Turismo académico y científico (cursos, idiomas, etc.).\n7. Turismo de naturaleza (ecoturismo, agroturismo, vacaciones en la naturaleza).\n8. Turismo de salud (belleza, salud mental y corporal, hospitalario y no hospitalario).\n9. Turismo de negocios.\n10. Viajes de incentivo.\n11. Turismo de cruceros.\n12. Turismo de intereses sociales.\n13'